# Example code to decode EMG markers files

This notebook shows example code to decode markers files after EMG onset and offset detection. 

To use this code, event markers need to be stored in myonset Events structure. Using events from continuous data file is recommended (i.e., time is relative to the beginning of the EMG recording file). However, some options are available if you want to use EpochEvents object (i.e., time events is relative to the beginning of each trial). In this case, replace the function 'events_to_df' by 'epochevents_to_df'. 

This code assumed that your data can be 'epoched', i.e., divided in a succession of trials. It cannot be used as is to decode event markers in data that cannot be divided into trials. 

Output of this code is two dataframes containing data from all markers files contained in folder 'path_mrk'. Each dataframe line corresponds to one single trial from one marker file. The two dataframes contain redundant data, the 'full' dataframe contains all columns used at this processing step, the 'compact' dataframe contains only columns usually required for next step analysis.


### Columns of compact dataframe:
- column 0 : dataframe index
- participant : participant name (defined by keys of pp_ins dictionnary) 
- group : group of this participant, defined in pp_group dictionnary
- compatibility : trial compatibility
- correct_resp : response expected for this stimulus (trial correct response)
- accuracy : 'correct' if actual response corresponds to correct response, 'incorrect' otherwise 
- color : color of the presented stimulus
- pos : position of the presented stimulus
- emg_type : EMG type name, corresponding to sequence_onset (described below) from wich response code is removed. The prefix 'pure' indicates that only one onset was present, the prefix 'partial' indicates that no response was present. EMG is 'unclassified' and a warning is printed if :
    1) some events are observed in onset sequence column after the response
    2) the last onset channel in onset sequence column does not match the response (e.g., accuracy is 'correct' but last onset is 'incorrect').
    
    To remove warnings, set 'print_warning' to False in function 'classify_emg' 
- sequence_onset : sequence of EMG onset(s) and response in that trial. 'C' means EMG onset on correct side , 'I' means EMG onset on incorrect side and 'R' means mechanical response
- rt : reaction time, time between stimulus and response events
- premotor_time : time between stimulus and onset of last EMG preceding the response, on responding side (i.e., EMG EMG supposed to cause the mechanical response)
- motor_time : time between onset of last EMG preceding the response, on responding side (i.e., EMG supposed to cause the mechanical response) and the mechanical response
- lat_first_partial : latency of first partial EMG between stimulus and first EMG onset on non-responding side 
- stim_time : time (in s) corresponding to the occurence of this trial stimulus event in the original marker file (i.e., in continuous data)
- onset_time : list of time onset(s) (in s) in this trial in the original marker file (i.e., in continuous data)
- onset_chan : list of channel onset(s) in this trial in the original marker file (i.e., in continuous data)
- offset_time : list of time offset(s) (in s) in this trial in the original marker file (i.e., in continuous data)
- offset_chan : list of channel offset(s) in this trial in the original marker file (i.e., in continuous data)

### Additional columns in full dataframe:
- stim_code : code of stimulus event in this trial 
- stim_sample : time (in sample) corresponding to the occurence of this trial stimulus event in the original marker file (i.e., in continuous data)
- resp_code : code of response event in this trial 
- resp_time	: time (in s) corresponding to the occurence of this trial response event in the original marker file (i.e., in continuous data)
- resp_sample : time (in sample) corresponding to the occurence of this trial response event in the original marker file (i.e., in continuous data)
- onset_code : list of code(s) of onset(s) event(s) in this trial
- onset_sample : list of time onset(s) (in sample) in this trial in the original marker file (i.e., in continuous data)	
- offset_code : list of code(s) of offset(s) event(s) in this trial
- offset_sample : list of time offset(s) (in sample) in this trial in the original marker file (i.e., in continuous data)	
- other_code : list of code(s) of other event(s) in this trial (i.e., not stimulus, response, onset or offset)
- other_chan : list of channel(s) of other event(s) in this trial (i.e., not stimulus, response, onset or offset)
- other_time : list of time (in s) of other event(s) in this trial (i.e., not stimulus, response, onset or offset)
- other_sample : list of time (in sample) of other event(s) in this trial (i.e., not stimulus, response, onset or offset)
- onset_correct : list of time onset(s) (in s) occuring on the correct side in this trial in the original marker file (i.e., in continuous data)
- offset_correct : list of time offset(s) (in s) occuring on the correct side in this trial in the original marker file (i.e., in continuous data)
- onset_incorrect : list of time onset(s) (in s) occuring on the incorrect side in this trial in the original marker file (i.e., in continuous data)
- offset_incorrect : list of time offset(s) (in s) occuring on the incorrect side in this trial in the original marker file (i.e., in continuous data)
- onset_resp : list of time onset (in s) occuring on the response side, closest but anterior to the response (i.e., supposed to be the EMG onset burst leading to the mechanical response), in this trial in the original marker file (i.e., in continuous data)
- offset_resp : list of time offset (in s) occuring on the response side, just after onset_resp (i.e., the end of the EMG burst leading to the mechanical response), in this trial in the original marker file (i.e., in continuous data)
- onset_non_resp : list of time onset(s) (in s) other than onset_resp in this trial in the original marker file (i.e., in continuous data)
- offset_non_resp : list of time offset(s) (in s) other than offset_resp in this trial in the original marker file (i.e., in continuous data)



In [ ]:
import os
import pandas as pd
import numpy as np
import myonset as myo
import eventsdf as evt

Set the path and list all files

In [ ]:
#path_mrk = os.path.join('.','corrected_detection')
path_mrk = os.path.join('.')

# where to save output xlsx file
path_output = os.path.join('.')

# name of output xlsx
fname_output = 'all_emg_markers.xlsx'


## List participants

In [ ]:
#list_pp = ['s1','s2']
list_pp = ['s1']


Set some variables -> you must fill in the instruction dictionnary for each new participant

In [ ]:
# list of participant numbers and associated instructions
pp_ins = {'s1' : 'red_left',\
          's2' : 'red_right',\
          }

Set pp_group if you have more than one group of participants

In [ ]:
pp_group = {'s1' : 'A',\
            's2' : 'B',\
            }


## Set events codes

### Stimulus events

Set stimulus events codes

In [ ]:
stim_id = {'red_left':    12,\
           'red_right':   22,\
           'green_left':  11,\
           'green_right': 21}

stim_codes = list(stim_id.values())

Set codes_t0 id (i.e., events used for segmentation, like stimulus or fixation cross)

In [ ]:
# by default, use stimulus codes
t0_codes = stim_codes

If necessary, set more attributes or stimulus codes (e.g., to compute compatibility)

In [ ]:
stim_pos = {11: 'left', 12: 'left', 21: 'right', 22: 'right'}
stim_color = {11: 'green', 12: 'red', 21: 'green', 22: 'red'}

### EMG and reponse events

Set response events codes

In [ ]:
resp_id = {'left':  100,\
           'right': 200}

resp_codes = list(resp_id.values())

Set EMG onset and offset events codes

In [ ]:
emg_id = {'onset':  131,\
          'offset': 132,\
          'peak':   133}

### Stimulus, response and EMG channels associations

Set correct stimulus-response association (may depend on the instuction)

In [ ]:
correct_stim_resp = {'red_left'  : {11: 100, 12: 200, 21: 100, 22: 200 },\
                     'red_right' : {11: 200, 12: 100, 21: 200, 22: 100}}


Set the response-emg_chan association

In [ ]:
resp_emg_chan = {100: 0 , 200: 1}

## Set signal sampling frequency

Either globally (if you have the same sampling frequency for all participants), or individually

In [ ]:
# global
#sf = 2048

# individual
sf = {'s1' : 2048,\
      's2' : 1024,\
      }

## Reads corrected events files and decodes events 

In [ ]:
full_df = pd.DataFrame()
for pp in list_pp :
    
    f = pp + '_corrected_evts.csv'
    print('Reading file {}'.format(f))

    pp_events = myo.load_continuous(os.path.join(path_mrk,f), sep=',', sf=sf[pp], col_sample=0, col_code=2, col_chan=3)

    pp_df = evt.events_to_df(pp_events, t0_codes, stim_codes, resp_codes,\
                             onset_codes=emg_id['onset'], offset_codes=emg_id['offset'])
   
    pp_df = evt.decode_accuracy(pp_df, correct_stim_resp[pp_ins[pp]],\
                                resp_emg_chan)
    pp_df = evt.classify_emg(pp_df)
    
    # include condition columns
    pp_df['participant'] = [pp]*pp_df.shape[0]
    pp_df['group'] = [pp_group[pp]]*pp_df.shape[0]
    
    for stim in stim_codes:
        pp_df.loc[pp_df['stim_code'] == stim, 'color'] = stim_color[stim]
        pp_df.loc[pp_df['stim_code'] == stim, 'pos'] = stim_pos[stim]
        pp_df.loc[pp_df['stim_code'] == stim, 'correct_resp'] = {v: k for k, v in resp_id.items()}[correct_stim_resp[pp_ins[pp]][stim]]
    
    # append to big table
    full_df = pd.concat((full_df,pp_df), ignore_index=True, sort=False)



Define trial compatibility by comparing stimulus position and correct response columns

In [ ]:
full_df.loc[full_df['pos'] == full_df['correct_resp'],'compatibility'] = 'comp'
full_df.loc[full_df['pos'] != full_df['correct_resp'],'compatibility'] = 'inc'

Compute latencies (reation time, premotor time, motor time, and latency of first EMG onset on non-repsonding side)

In [ ]:
full_df['rt'] = full_df['resp_time'] - full_df['stim_time']
full_df['premotor_time'] = np.array(myo.utils.utilsfunc.remove_list(full_df['onset_resp'])) - full_df['stim_time']
full_df['motor_time'] = full_df['resp_time'] - np.array(myo.utils.utilsfunc.remove_list(full_df['onset_resp']))

full_df['lat_first_partial'] = np.array(myo.utils.utilsfunc.remove_list(full_df['onset_non_resp'])) - full_df['stim_time']


Re-order columns

In [ ]:
useful_columns = ['participant','group','compatibility','correct_resp','accuracy','color','pos',\
                  'emg_type','sequence_onset','rt','premotor_time','motor_time','lat_first_partial',\
                  'stim_time','onset_time','onset_chan','offset_time','offset_chan']

other_columns = [ c for c in full_df.columns if c not in useful_columns]

In [ ]:
full_df = full_df[useful_columns + other_columns]

Create a compact df with only useful columns

In [ ]:
compact_df = full_df[useful_columns]

Save both full and compact tables

In [ ]:
full_df.to_excel(os.path.join(path_output,fname_output[:-5]+'_full'+fname_output[-5:]))    
compact_df.to_excel(os.path.join(path_output,fname_output[:-5]+'_compact'+fname_output[-5:]))  